In [ ]:
# Cell 1: Imports & Environment Setup
%pip install regex pandas numpy tqdm openpyxl soynlp
%pip install torch>=2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install sentence-transformers FlagEmbedding
import os
import sys
import logging
import torch
import openpyxl
import regex as re
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Any, Optional, Callable
from tqdm.notebook import tqdm
from FlagEmbedding import BGEM3FlagModel
from transformers import AutoTokenizer, AutoModel

# 파일 경로 설정
input_path = "C:/Users/junto/Downloads/head-repo/SP/split_p/input_p.xlsx"
output_path = "C:/Users/junto/Downloads/head-repo/SP/split_p/output_p.xlsx"

# (Optional) 환경 점검
print("NumPy", np.__version__)
print("Pandas", pd.__version__)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
NumPy 2.2.6
Pandas 2.3.0


In [5]:
# Cell 2: Tokenizer Initialization
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

def tokenize(text, method='bert'):
    if method == 'bert':
        return bert_tokenizer.tokenize(text)
    elif method == 'spm':
        return sp.encode(text, out_type=str)
    else:
        raise ValueError(f"Unknown tokenization method: {method}")
    
# Cell 3: Load Source & Target Texts from Excel
# input_p.xlsx에 'src' 및 'tgt' 컬럼이 있다고 가정
df_input = pd.read_excel(input_path)
src_full = df_input['원문'].astype(str).str.cat(sep=' ')
tgt_full = df_input['번역문'].astype(str).str.cat(sep=' ')

In [15]:
# Cell 4: Sentence Splitting (기존 기준 유지)
def split_sentences(text):
    sents = re.split(r'(?<=[。？！.!?])\s+', text)
    parts = []
    for s in sents:
        if len(s) > 150:
            parts.extend([s[i:i+150] for i in range(0, len(s), 150)])
        else:
            parts.append(s)
    merged, buffer = [], ''
    for seg in parts:
        han_count = len(re.findall(r'[\u4E00-\u9FFF]', seg))
        if han_count <= 3:
            buffer += seg
        else:
            if buffer:
                if merged:
                    merged[-1] += buffer
                buffer = ''
            merged.append(seg)
    if buffer and merged:
        merged[-1] += buffer
    return [m.strip() for m in merged]

In [19]:
# Cell 5: Generate Target Units (Tokenized)
tgt_sents = split_sentences(tgt_full)
tgt_units = [' '.join(tokenize(s, method='bert')) for s in tgt_sents]

# Cell 6: Initial Source Chunking
def chunk_src(src_text, max_chars=200):
    sents = split_sentences(src_text)
    chunks, buf = [], ''
    for s in sents:
        if len(buf) + len(s) > max_chars:
            chunks.append(buf.strip())
            buf = s
        else:
            buf += s
    if buf:
        chunks.append(buf.strip())
    return chunks

src_chunks = chunk_src(src_full)

In [23]:
# OpenAI Embeddings API 사용
import openai
import numpy as np
from typing import List
import time
from openai import OpenAI

# OpenAI API 키 설정 (필요한 경우)
client = OpenAI(api_key='your-api-key')

def get_openai_embeddings(texts: List[str], model="text-embedding-3-small"):
    """OpenAI API를 사용한 임베딩"""
    embeddings = []
    
    for i, text in enumerate(texts):
        try:
            response = openai.embeddings.create(
                input=text,
                model=model
            )
            embeddings.append(response.data[0].embedding)
            
            if i % 10 == 0:
                print(f"Processed {i}/{len(texts)} texts")
                time.sleep(0.1)  # API 제한 방지
                
        except Exception as e:
            print(f"Error with text {i}: {e}")
            # 기본값으로 0 벡터 사용
            embeddings.append([0.0] * 1536)  # text-embedding-3-small 차원
    
    return np.array(embeddings)

# 사용 예시 (API 키가 있는 경우)
# src_embs = get_openai_embeddings(src_chunks)
# tgt_embs = get_openai_embeddings(tgt_units)

In [25]:
# Cell 9: Split & Collate Results from assignments (OpenAI Embedding 기반)
records = []
for tgt_i, src_j in assignments:
    records.append({
        'tgt_id': tgt_i,
        'tgt_text': tgt_sents[tgt_i],
        'src_chunk_id': src_j,
        'src_text': src_chunks[src_j]
    })

df_output = pd.DataFrame(records)
df_output.head()

NameError: name 'assignments' is not defined

In [ ]:
# Cell 10: Save Output to Excel
df_output.to_excel(output_path, index=False)
print(f"Output saved to {output_path}")